In [18]:

import osmnx as ox
import folium
import pandas as pd
import numpy as np
import utils
import folium_utils
import networkx as nx
import googlemaps
import os
from dotenv import load_dotenv
import key_store
load_dotenv()

# from IPython.display import display
# pd.set_option('display.max_rows', None)
                                
latitude_start = 35.330878
longitude_start = 136.951774
latitude_end = 35.402261
longitude_end = 137.072889

# キャッシュを使う
ox.config(use_cache=True, log_console=True)

graph = ox.graph_from_bbox(north=max(latitude_start, latitude_end),
                           south=min(latitude_start, latitude_end),
                           east=max(longitude_start, longitude_end),
                           west=min(longitude_start, longitude_end),
                           network_type='drive',
                           simplify=True,
                           retain_all=True,
                           custom_filter='["highway"~"secondary|secondary_link|primary|primary_link|trunk|trunk_link"]["lanes"=2]')
graph2 = ox.graph_from_bbox(north=max(latitude_start, latitude_end),
                           south=min(latitude_start, latitude_end),
                           east=max(longitude_start, longitude_end),
                           west=min(longitude_start, longitude_end),
                           network_type='drive',
                           simplify=True,
                           retain_all=True,
                           custom_filter='["highway"="tertiary"]')
graph = nx.compose(graph, graph2)
# graph = ox.graph_from_bbox(north=max(latitude_start, latitude_end),
#                            south=min(latitude_start, latitude_end),
#                            east=max(longitude_start, longitude_end),
#                            west=min(longitude_start, longitude_end),
#                            network_type='drive',
#                            simplify=True,
#                            retain_all=True,
#                            custom_filter='["highway"~"secondary|secondary_link|primary|primary_link|trunk|trunk_link|tertiary"]')

# グラフデータをGeoDataFrameに変換  
gdf_nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False)
gdf_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)

# 3座標間の角度を求める
def calculate_angle_between_vectors(A, B, C):
    vector_AB = np.array(B) - np.array(A)
    vector_BC = np.array(C) - np.array(B)
    
    dot_product = np.dot(vector_AB, vector_BC)
    norm_AB = np.linalg.norm(vector_AB)
    norm_BC = np.linalg.norm(vector_BC)
    
    cosine_theta = dot_product / (norm_AB * norm_BC)
    angle_rad = np.arccos(cosine_theta)
    angle_deg = np.degrees(angle_rad)
    return angle_deg

# 開始位置列を追加する
gdf_edges['start_point'] = gdf_edges['geometry'].apply(lambda x: x.coords[0])
# 終了位置列を追加する
gdf_edges['end_point'] = gdf_edges['geometry'].apply(lambda x: x.coords[-1])

# 逆方向のベクトルを持つエッジを削除する
gdf_edges = utils.drop_duplicate_edge(gdf_edges)



graph_all = ox.graph_from_bbox(north=max(latitude_start, latitude_end),
                           south=min(latitude_start, latitude_end),
                           east=max(longitude_start, longitude_end),
                           west=min(longitude_start, longitude_end),
                           network_type='drive',
                           simplify=True,
                           retain_all=True)
all_nodes = ox.graph_to_gdfs(graph_all, nodes=True, edges=False)
# エッジ内の分岐数を取得する
for index, row in gdf_edges.iterrows():
  # ジオメトリーの座標と一致するノードを取得する
  nodes = all_nodes[all_nodes.geometry.intersects(row.geometry)]
  # 進行方向と逆方向のノードを除外して分岐数を計算する
  gdf_edges.at[index, 'all_street_cnt'] = nodes['street_count'].sum() - (len(nodes) * 2)
  gdf_edges.at[index, 'node_cnt'] = len(nodes)




# 座標間の角度の変化の合計値を求める
gdf_edges['geometory_angle_total'] = gdf_edges['geometry'].apply(
    lambda x: sum([calculate_angle_between_vectors(x.coords[i-1], x.coords[i], x.coords[i+1]) for i in range(1, len(x.coords)-1)])
)

# 以下の条件に該当するエッジを峠の候補にする
# エッジの距離が1000m以上
# ジオメトリーの座標感の角度変化の合計が120度以上
# 1500mで10分岐以下
lower_bound_meter = 1000
branch_meter_rate = 0.0066 # 1500mで10分岐以上ある場合は対象外にする
gdf_edges['is_target'] = np.where((gdf_edges['length'] >= lower_bound_meter) & (gdf_edges['geometory_angle_total'] > 120) & (gdf_edges['all_street_cnt'] / gdf_edges['length'] <= branch_meter_rate), 1, 0)

# targe_edgesを並び替える
target_edges = gdf_edges[gdf_edges['is_target'] == 1]
target_edges = target_edges.sort_values(['geometory_angle_total'], ascending=[False])

# 高度の変化を激しい道を優先する
# クライアントの作成
gmaps = googlemaps.Client(key=os.environ['GOOGLE_API_KEY'])
for index, row in target_edges.iterrows():
    locations = list(row.geometry.coords)
    # locationsの緯度と経度の要素番号を逆にする
    locations = [list(reversed(location)) for location in locations]
    elevations = []
    # 1度のリクエストで512箇所までしか指定できないので分割してリクエストを送る
    for i in range(0, len(locations), 512):
        # apiの呼び出し回数を減らすためにキャッシュさせる    
        parameter = locations[i:i+512]
        value = key_store.load(parameter)
        if value:
            print('store tukatta')
            print(value)
            elevations.append(value)
        else:
            elevations.extend(gmaps.elevation(parameter))
            key_store.save(parameter, elevations)

    # elevationsのevelationの差の合計を求める
    elevation_diff_total = 0
    target_edges.at[index, 'height_diff_total'] = 0
    for i in range(1, len(elevations)):
        target_edges.at[index, 'height_diff_total'] += abs(elevations[i]['elevation'] - elevations[i-1]['elevation'])
    ## 距離あたりの高低差の変化率を求める
    target_edges.at[index, 'height_diff_rate'] = target_edges.at[index, 'height_diff_total'] / row.length

#   # ジオメトリーの座標と一致するノードを取得する
#   nodes = all_nodes[all_nodes.geometry.intersects(row.geometry)]
#   # 進行方向と逆方向のノードを除外して分岐数を計算する
#   gdf_edges.at[index, 'all_street_cnt'] = nodes['street_count'].sum() - (len(nodes) * 2)
#   gdf_edges.at[index, 'node_cnt'] = len(nodes)

# height_diff_rateとheight_diff_totalの積を求める
target_edges['rate'] = target_edges['height_diff_rate'] * target_edges['geometory_angle_total']
target_edges = target_edges.sort_values(['rate'], ascending=[False])

target_edges['google_map'] = ''
for index, row in target_edges.iterrows():
    target_edges.at[index, 'google_map'] =  f"https://www.google.co.jp/maps/dir/{row.start_point[1]},{row.start_point[0]}/'{row.end_point[1]},{row.end_point[0]}'"

# 地図を表示する
map_osm = ox.plot_graph_folium(graph, edge_width=2)

folium_utils.add_marker(map_osm, latitude_start, longitude_start, "st", "red")
folium_utils.add_marker(map_osm, latitude_end, longitude_end, "ed", "green")

# # is_targetが正のエッジにplotを立てて緯度と経度を表示する
# for index, row in gdf_edges[gdf_edges['is_target'] == 1].iterrows():
#     folium_utils.add_marker(map_osm,
#                             row['geometry'].coords[0][1],
#                             row['geometry'].coords[0][0],
#                             f"{row['geometry'].coords[0][1]}, {row['geometry'].coords[0][0]}, {row['geometory_angle_total']}, {row['length']}",
#                             "red")

# 候補を全て表示
map_osm.add_child(
    folium.features.GeoJson(
        gdf_edges[gdf_edges['is_target'] == 1].to_json(),
        # https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html
        style_function=lambda x: {
            'color': '#00ff00',
            'fill_opacity': 0,
            'weight': 2,
        }
    )
    .add_child(folium.features.GeoJsonPopup(
        fields=['start_point', 'end_point', 'geometory_angle_total', 'length', 'all_street_cnt', 'node_cnt'],
        aliases=['start_point', 'end_point', 'geometory_angle_total', 'length', 'all_street_cnt', 'node_cnt'],
        localize=True
    ))
)

# 候補の上位10件を表示
map_osm.add_child(
    folium.features.GeoJson(
        target_edges.head(10).to_json(),
        # https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html
        style_function=lambda x: {
            'color': '#ff0000',
            'fill_opacity': 0,
            'weight': 5,
        }
    )
    # 緯度と経度を表示する
    .add_child(folium.features.GeoJsonPopup(
        fields=['start_point', 'end_point', 'geometory_angle_total', 'length', 'all_street_cnt', 'node_cnt', 'rate', 'google_map'],
        aliases=['start_point', 'end_point', 'geometory_angle_total', 'length', 'all_street_cnt', 'node_cnt', 'rate', 'google_map'],
        localize=True
    ))
)

# # 上位10件のジオメトリーをjsonで出力する
# target = target_edges
# # geometryとgeometory_angle_totalのみを抽出する
# target = target[['geometry', 'geometory_angle_total']]
# # LINESTRINGを緯度と経度のリストに変換する
# target['geometry'] = target['geometry'].apply(lambda x: list(x.coords))
# # jsonに変換する
# target[['geometry', 'geometory_angle_total']].to_json('./html/target.json', orient='records')

/tmp/ipykernel_4273/1285748274.py:23: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)
/tmp/ipykernel_4273/1285748274.py:92: PerformanceWarning: indexing past lexsort depth may impact performance.
  gdf_edges.at[index, 'all_street_cnt'] = nodes['street_count'].sum() - (len(nodes) * 2)
/tmp/ipykernel_4273/1285748274.py:93: PerformanceWarning: indexing past lexsort depth may impact performance.
  gdf_edges.at[index, 'node_cnt'] = len(nodes)


store tukatta
[{'elevation': 184.6412963867188, 'location': {'lat': 35.35336, 'lng': 137.05321}, 'resolution': 9.543951988220215}, {'elevation': 184.3439025878906, 'location': {'lat': 35.35353, 'lng': 137.05312}, 'resolution': 9.543951988220215}, {'elevation': 186.3815612792969, 'location': {'lat': 35.35386, 'lng': 137.05302}, 'resolution': 9.543951988220215}, {'elevation': 186.4642486572266, 'location': {'lat': 35.35393, 'lng': 137.05296}, 'resolution': 9.543951988220215}, {'elevation': 186.2644958496094, 'location': {'lat': 35.35398, 'lng': 137.05281}, 'resolution': 9.543951988220215}, {'elevation': 185.6983337402344, 'location': {'lat': 35.35401, 'lng': 137.05266}, 'resolution': 9.543951988220215}, {'elevation': 185.4567565917969, 'location': {'lat': 35.35404, 'lng': 137.05253}, 'resolution': 9.543951988220215}, {'elevation': 184.8324432373047, 'location': {'lat': 35.35413, 'lng': 137.05241}, 'resolution': 9.543951988220215}, {'elevation': 185.1996307373047, 'location': {'lat': 35.3

/tmp/ipykernel_4273/1285748274.py:138: PerformanceWarning: indexing past lexsort depth may impact performance.
  target_edges.at[index, 'height_diff_total'] = 0
/tmp/ipykernel_4273/1285748274.py:142: PerformanceWarning: indexing past lexsort depth may impact performance.
  target_edges.at[index, 'height_diff_rate'] = target_edges.at[index, 'height_diff_total'] / row.length
/tmp/ipykernel_4273/1285748274.py:159: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  map_osm = ox.plot_graph_folium(graph, edge_width=2)
